In [3]:
import pandas as pd
df = pd.read_csv("takehome_users.csv",encoding='latin-1')
activity =  pd.read_csv("takehome_user_engagement.csv")

df["last_session_creation_time"] = pd.to_datetime(df["last_session_creation_time"])

activity['time_stamp2'] = pd.to_datetime(activity['time_stamp'])

In [4]:
repeated_users=activity.groupby('user_id').filter(lambda x: len(x) >= 3)


In [5]:
grouped_users = repeated_users.groupby('user_id')


In [17]:
def active_users(period, days_logged, user):
	visited = len(user.index)
	i, count = 0, 1
	active_user = False
	while count < days_logged:		
		if (i+2) < visited:
			if (user['time_stamp2'].iloc[i + 1] - user['time_stamp2'].iloc[i]) <= pd.Timedelta(days=period) and (user['time_stamp2'].iloc[i + 1] - user['time_stamp2'].iloc[i]) > pd.Timedelta(days=1) :
				count += 1
				new_timeframe = (user['time_stamp2'].iloc[i + 1] - user['time_stamp2'].iloc[i])
				if (user['time_stamp2'].iloc[i + 2] - user['time_stamp2'].iloc[i + 1]) <= new_timeframe and (user['time_stamp2'].iloc[i + 2] - user['time_stamp2'].iloc[i + 1]) > pd.Timedelta(days=1):
					active_user = True
					count += 1
				else: 
					i += 1
					count = 1
			else:
				i += 1
				count = 1
		else:
			count = days_logged
	return active_user

In [19]:
unique_active  = grouped_users.filter(lambda x: active_users(period=7, days_logged=3, user=x) ==True)

In [28]:
unique_active = pd.DataFrame(pd.Series.unique(unique_active['user_id']))

In [42]:
unique_active.columns =['user_id']
unique_active['adopted_user'] = 1


,user_id,adopted_user
0,2,1
1,10,1
2,33,1
3,42,1
4,50,1
...,...,...
1610,11969,1
1611,11975,1
1612,11980,1
1613,11988,1


In [41]:
adopted_user_info = pd.merge(unique_active, df, how='outer',left_on='user_id', right_on='object_id')

In [44]:
adopted_user_info['adopted_user'] = adopted_user_info['adopted_user'].fillna(0)

In [45]:
adopted_user_info.size

144000

In [46]:
len(adopted_user_info)

12000